In [ ]:
import wandb
from collections import defaultdict
import pandas as pd
import numpy as np

import random

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
from IPython.display import display_html
import os

%load_ext autoreload
%autoreload 2

In [ ]:
api = wandb.Api()

In [ ]:
SAVE_DIR = "/Users/pari/Desktop/"
#/Users/pari/Desktop

In [ ]:
pgruns = api.runs("pari/MyCEB",
    {"$and": [
       {"config.algs": "postgres"},
       {"config.val_size":0},
       {"config.test_size":0},
       {"config.query_templates":"all"},
       {"config.query_dir":"queries/imdb-unique-plans"},
       {"config.num_samples_per_template":-1},
       {"$or": [
           {"tags":"v14"},
       ]}
    ]
    })

print(f"Found {len(pgruns)} runs")

In [ ]:
import time
start = time.time()
#summary_list, config_list, name_list = [], [], []
#TAGS = ["baselines", "baseline", "best_model_0.2", "best_model"]
pgdfs = []

for run in pgruns:
    skip = False
    if skip:
        continue
    if run.State != "finished":
        continue
    
    data = defaultdict(list)
    data["Tags"].append(run.Tags)
    data["name"].append(run.name)
    
    
    for k,v in run.config.items():
        if not k.startswith("_") or not k.contains("/"):
            data[k].append(v)

    for k,v in run.summary._json_dict.items():
        if k.startswith("_"):
            continue
        if "/" in k:
            continue
        data[k].append(v)
        
    pgdfs.append(pd.DataFrame(data))

print("took: ", time.time()-start)

In [ ]:
pgdf = pd.concat(pgdfs)

In [ ]:
pgdf

In [ ]:
#pgdf = pgdf[~pgdf["Final-Relative-TotalPPCost-train-2a"].isna()]

In [ ]:
runs = api.runs("pari/MyCEB",
    {"$and": [
       {"config.algs": "mscn"},
       {"config.train_test_split_kind":"custom"},
       {"config.test_tmps":"all"},
       {"config.query_dir":"queries/imdb-unique-plans"},
       {"config.no_regex_templates":1},
       {"config.embedding_fn":"none"},
       {"config.feat_separate_alias":0},
       {"$or": [ 
           {"config.subplan_level_outputs":0},
           {"config.subplan_level_outputs":None},
       ]},
       {"$or": [ 
           {"config.training_opt":""},
           {"config.training_opt":"none"},
           {"config.training_opt":None},
       ]},
       {"config.weight_decay":0},
       {"$or": [
           {"tags":"v15"},
       ]}
    ]
    })

print(f"Found {len(runs)} runs")

In [ ]:
import time
start = time.time()
#summary_list, config_list, name_list = [], [], []
#TAGS = ["baselines", "baseline", "best_model_0.2", "best_model"]
dfs = []

for run in runs:
    skip = False
    if skip:
        continue
    if run.State != "finished":
        continue
    
    data = defaultdict(list)
    data["Tags"].append(run.Tags)
    data["name"].append(run.name)
    
    
    for k,v in run.config.items():
        if not k.startswith("_") or not k.contains("/"):
            data[k].append(v)

    for k,v in run.summary._json_dict.items():
        if k.startswith("_"):
            continue
        if "/" in k:
            continue
        data[k].append(v)
        
    dfs.append(pd.DataFrame(data))

print("took: ", time.time()-start)

In [ ]:
df = pd.concat(dfs)

In [ ]:
print(len(df))

In [ ]:
print(set(df["table_features"]))
print(set(df["join_features"]))
print(set(df["set_column_feature"]))
print(set(df["max_discrete_featurizing_buckets"]))
#print(set(df["mask_unseen_subplans"]))
print(set(df["onehot_dropout"]))
#print(set(df["onehot_mask_truep"]))
print(set(df["loss_func_name"]))
print(set(df["hidden_layer_size"]))

In [ ]:
def get_row_featurization(row):
    ret = ""
    if row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["loss_func_name"] == "mse" \
        and row["ynormalization"] == "log" \
        and row["load_query_together"] == 0 \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] == 0:
        if row["sample_bitmap"] == 0 and row["max_discrete_featurizing_buckets"] == 10 and \
            row["max_like_featurizing_buckets"] == 10:
            ret = "Default"
        elif row["sample_bitmap"] == 1:
            ret = "Default + Sample Bitmap-" \
            + str(row["max_discrete_featurizing_buckets"]) \
            + str(row["max_like_featurizing_buckets"])
        else:
            ret = "Default (unknown)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["max_discrete_featurizing_buckets"] == 1 \
        and row["max_like_featurizing_buckets"] == 1 \
        and row["loss_func_name"] == "mse" \
        and row["load_query_together"] == 0 \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] == 0:
        ret = "Default (onehot, no-buckets)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "mse" \
        and row["load_query_together"] == 0 \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] == 2 \
        and row["heuristic_features"] == 1:
        ret = "Default-dropout-" + str(row["onehot_mask_truep"])
    elif row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "flowloss" \
        and row["weight_decay"] == 0 \
        and row["normalize_flow_loss"] == 1 \
        and row["flow_features"] == 1:
        ret = "FlowLoss (onehot)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] == "onehot-stats" \
        and row["join_features"] == "onehot-stats" \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "flowloss" \
        and row["weight_decay"] == 0 \
        and row["normalize_flow_loss"] == 1 \
        and row["flow_features"] == 1:
        ret = "FlowLoss (onehot-stats)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] == "onehot-stats" \
        and row["join_features"] == "onehot-stats" \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "flowloss" \
        and row["weight_decay"] == 0 \
        and row["normalize_flow_loss"] == 1 \
        and row["flow_features"] == 1 \
        and row["onehot_dropout"] == 2:
        ret = "FlowLoss (onehot-stats-dropout)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "flowloss" \
        and row["weight_decay"] == 0 \
        and row["normalize_flow_loss"] == 0 \
        and row["flow_features"] == 1:
        ret = "FlowLoss2 (onehot)"
    elif row["table_features"] == 1 \
        and row["set_column_feature"] in ["1", 1, "onehot"] \
        and row["join_features"] in ["1", 1, "onehot"] \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "mse+ranknet" \
        and row["load_query_together"] == 1:
        ret = "MSE+ranknet"
    
    elif row["table_features"] == 1 \
        and row["set_column_feature"] == "onehot-stats" \
        and row["join_features"] == "onehot-stats" \
        and row["loss_func_name"] == "mse" \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] == 2:
        if row["sample_bitmap"] == 1:
            ret = "Onehot-Stats-Dropout-Bitmap-" + \
            str(row["max_discrete_featurizing_buckets"]) + "-" + \
            str(row["max_like_featurizing_buckets"]) + "-" + \
            str(row["max_epochs"]) + "-" + \
            str(row["onehot_mask_truep"])
        else:
            ret = "Onehot-Stats-Dropout-" + str(row["max_discrete_featurizing_buckets"]) + "-" +\
            str(row["max_like_featurizing_buckets"]) + "-" + \
            str(row["max_epochs"]) + "-" + \
            str(row["onehot_mask_truep"])
        
    elif row["table_features"] == 1 \
        and row["set_column_feature"] == "onehot" \
        and row["join_features"] == "onehot" \
        and row["loss_func_name"] == "mse" \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] == 2:
        if row["sample_bitmap"] == 1:
            ret = "Onehot-Dropout-Bitmap-" + \
            str(row["max_discrete_featurizing_buckets"]) + "-" + \
            str(row["max_like_featurizing_buckets"]) + "-" + \
            str(row["max_epochs"]) + "-" + \
            str(row["onehot_mask_truep"])
        else:
            ret = "Onehot-Dropout-" + str(row["max_discrete_featurizing_buckets"]) + "-" +\
            str(row["max_like_featurizing_buckets"]) + "-" + \
            str(row["max_epochs"]) + "-" + \
            str(row["onehot_mask_truep"])
        
    elif row["table_features"] == 1 \
        and row["set_column_feature"] == "onehot-stats" \
        and row["join_features"] == "onehot-stats" \
        and row["max_discrete_featurizing_buckets"] == 10 \
        and row["max_like_featurizing_buckets"] == 10 \
        and row["loss_func_name"] == "mse" \
        and row["weight_decay"] == 0 \
        and row["onehot_dropout"] > 0 \
        and row["max_epochs"] == 20:
        ret = "Onehot-Stats-dropout" + str(row["onehot_dropout"])
    else:
        ret = "unknown"
    
    if row["feat_separate_like_ests"] == 1:
        ret += "-sepLike"
    
    if row["feat_true_base_cards"] == 1:
        ret += "-trueBase"
        
    if row["max_discrete_featurizing_buckets"] == 30:
        ret += "-bins30"
        
    if row["early_stopping"]:
        ret += "-ES" + str(row["early_stopping"])
    
    return ret

In [ ]:
df["Featurization"] = df.apply(lambda x: get_row_featurization(x), axis=1)
df.groupby(["Featurization"]).count()

In [ ]:
#resdata = defaultdict()
c1reskeys = []
c2reskeys = []
for key in df.keys():
    if 'Final-Relative-TotalPPCost-' in key:
        if "train" in key:
            continue
        if 'Final-Relative-TotalPPCost-val' == key:
            continue
        if "C2" in key:
            c2reskeys.append(key)
        else:
            c1reskeys.append(key)
print(len(c2reskeys), len(c1reskeys))

In [ ]:
#df2 = df[["train_tmps"] + c1reskeys]

In [ ]:
c1reskeys

In [ ]:
resdata = defaultdict(list)
for idx,row in df.iterrows():
    if row["train_tmps"] == "4a":
        continue
        
    resdata["Featurization"].append(row["Featurization"])
    resdata["Train"].append(row["train_tmps"])
    for k in c1reskeys:
        if "4a" in k:
            continue
        if "job" in k:
            continue
        if np.isnan(row[k]):
            continue
        if "val-" in k:
            tmp_name = k[k.rfind("-")+1:]
            assert tmp_name == row["train_tmps"]
            resdata[row["train_tmps"]].append(row[k])
        else:
            tmp_name = k[k.rfind("-")+1:]
            resdata[tmp_name].append(row[k])

In [ ]:
pgc1 = {}
for k in c1reskeys:
    k = k.replace("test", "train")
    if "job" in k:
        continue
    if "train" not in k:
        continue
    tmp = pgdf[~pgdf[k].isna()]
    if np.isnan(tmp[k].mean()):
        continue
    #print(k, tmp[k].mean(), tmp[k].min())
    tmp_name = k[k.rfind("-")+1:]
    pgc1[tmp_name] = tmp[k].mean()

In [ ]:
resdf = pd.DataFrame(resdata)

In [ ]:
# for k in resdf.keys():
#     if k in pgc1:
#         resdf[k] /= pgc1[k]

In [ ]:
set(resdf["Featurization"])

In [ ]:
print(set(resdf["Featurization"]))
rdf2 = resdf[resdf["Featurization"].isin(['Default-ES1', 'Default-ES2', 'Default-dropout-0.8-ES1'])]

In [ ]:
rdf2.groupby(["Featurization", "Train"]).mean().round(2)

In [ ]:
KEYS = ["1a", "2a", "2b", "2c", "3a", "5a", "6a", "7a", "8a"]
rd3 = rdf2[KEYS] > 2
print(rd3.sum())
rd3 = rdf2[KEYS] < 1
print(rd3.sum())

In [ ]:
sns.barplot(data=rdf2, x="Train", y="test", hue="Featurization")